In [6]:
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.models import *
from keras.layers import *
from keras.layers.core import Dense
from keras.utils import plot_model


In [7]:
data = pd.read_csv("non-trans_10000_5.csv") 
df = pd.DataFrame(data)

In [8]:
(trainX, testX) = train_test_split(df, test_size=0.2, random_state=42)
trainY = trainX['picked_node']
testY = testX['picked_node']


# Iteration 1: one hot encoded

In [9]:
def process_structured_data(df, train, test):

    continuous = ['left_node', 'right_node']
    cs = MinMaxScaler()
    trainX = cs.fit_transform(train[continuous])
    testX = cs.transform(test[continuous])
    
    colourBin = LabelBinarizer().fit(df['left_colour'])
    trainLcolourX = colourBin.transform(train['left_colour'])
    testLcolourX = colourBin.transform(test['left_colour'])
    trainRcolourX = colourBin.transform(train['right_colour'])
    testRcolourX = colourBin.transform(test['right_colour'])
    trainX = np.hstack([trainX, trainLcolourX, trainRcolourX])
    testX = np.hstack([testX, testLcolourX, testRcolourX])
    
    return (trainX, testX)

In [10]:
(proc_trainX, proc_testX) = process_structured_data(df, trainX, testX)

In [11]:
def create_mlp(dim, regularizer=None):
    """Creates a simple two-layer MLP with inputs of the given dimension"""
    model = Sequential()
    model.add(Dense(8, input_dim=dim, activation="relu", kernel_regularizer=regularizer))
    model.add(Dense(1, activation="sigmoid", kernel_regularizer=regularizer))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [12]:
one_model = create_mlp(proc_trainX.shape[1])

In [13]:
one_model.fit(proc_trainX, trainY, batch_size=16, epochs=10, validation_data=(proc_testX, testY))

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 0s 58us/step - loss: 0.6473 - accuracy: 0.6258 - val_loss: 0.5868 - val_accuracy: 0.6900
Epoch 2/10
8000/8000 [==============================] - 0s 39us/step - loss: 0.5078 - accuracy: 0.7869 - val_loss: 0.4252 - val_accuracy: 0.8885
Epoch 3/10
8000/8000 [==============================] - 0s 41us/step - loss: 0.3600 - accuracy: 0.9275 - val_loss: 0.3037 - val_accuracy: 0.9470
Epoch 4/10
8000/8000 [==============================] - 0s 41us/step - loss: 0.2575 - accuracy: 0.9607 - val_loss: 0.2192 - val_accuracy: 0.9735
Epoch 5/10
8000/8000 [==============================] - 0s 39us/step - loss: 0.1888 - accuracy: 0.9759 - val_loss: 0.1678 - val_accuracy: 0.9830
Epoch 6/10
8000/8000 [==============================] - 0s 39us/step - loss: 0.1478 - accuracy: 0.9837 - val_loss: 0.1352 - val_accuracy: 0.9900
Epoch 7/10
8000/8000 [==============================] - 0s 40us/step - loss: 0.121

# Iteration 2: ordinal encoded

In [14]:
def process_integer_data(df, train, test):
    continuous = ['left_node', 'right_node']
    cs = MinMaxScaler()
    trainX = cs.fit_transform(train[continuous])
    testX = cs.transform(test[continuous])
    
    colourencoder = OrdinalEncoder().fit(np.array(df['left_colour']).reshape(-1,1))
    trainLcolourX = colourencoder.transform(np.array(train['left_colour']).reshape(-1,1))
    testLcolourX = colourencoder.transform(np.array(test['left_colour']).reshape(-1,1))
    trainRcolourX = colourencoder.transform(np.array(train['right_colour']).reshape(-1,1))
    testRcolourX = colourencoder.transform(np.array(test['right_colour']).reshape(-1,1))
    trainX = np.hstack([trainX, trainLcolourX, trainRcolourX])
    testX = np.hstack([testX, testLcolourX, testRcolourX])
    
    return (trainX, testX)

In [15]:
(proc_trainX, proc_testX) = process_integer_data(df, trainX, testX)

In [16]:
int_model = create_mlp(proc_trainX.shape[1])

In [17]:
int_model.fit(proc_trainX, trainY, batch_size=10, epochs=10, validation_data=(proc_testX, testY))

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 71us/step - loss: 0.7008 - accuracy: 0.5450 - val_loss: 0.6814 - val_accuracy: 0.5765
Epoch 2/10
8000/8000 [==============================] - 0s 60us/step - loss: 0.6751 - accuracy: 0.5775 - val_loss: 0.6696 - val_accuracy: 0.5605
Epoch 3/10
8000/8000 [==============================] - 0s 60us/step - loss: 0.6612 - accuracy: 0.5934 - val_loss: 0.6604 - val_accuracy: 0.6030
Epoch 4/10
8000/8000 [==============================] - 0s 60us/step - loss: 0.6529 - accuracy: 0.6140 - val_loss: 0.6527 - val_accuracy: 0.6165
Epoch 5/10
8000/8000 [==============================] - 0s 61us/step - loss: 0.6456 - accuracy: 0.6246 - val_loss: 0.6455 - val_accuracy: 0.6285
Epoch 6/10
8000/8000 [==============================] - 0s 61us/step - loss: 0.6389 - accuracy: 0.6310 - val_loss: 0.6396 - val_accuracy: 0.6400
Epoch 7/10
8000/8000 [==============================] - 0s 61us/step - loss: 0.632

# Iteration 3: Embedded

In [18]:
(proc_trainX, proc_testX) = process_integer_data(df, trainX, testX)

In [20]:
def emb(n):

    embedding_size = int(np.ceil(n/2))
    vocab  = n + 1

    model1 = Sequential()
    model1.add(Embedding(vocab ,embedding_size, input_length=1))
    model1.add(Reshape(target_shape=(embedding_size,)))

    model2 = Sequential()
    model2.add(Embedding(vocab ,embedding_size, input_length=1))
    model2.add(Reshape(target_shape=(embedding_size,)))

    model_rest = Sequential()
    model_rest.add(Dense(2, input_shape=(2,)))

    combined = concatenate([model_rest.output, model1.output, model2.output])

    x = Dense(8, activation="relu", kernel_initializer='he_normal')(combined)
    x = Dense(1, activation="sigmoid")(x)

    full_model = Model(inputs=[model_rest.input, model1.input, model2.input], outputs = x)
    full_model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer='adam')
    
    return full_model

emb_model = emb(len(np.unique(proc_trainX[:, 2])))

continous = proc_trainX[:, :2]
cat1 = proc_trainX[:, 2]
cat2 = proc_trainX[:, 3]

tcontinous = proc_testX[:, :2]
tcat1 = proc_testX[:, 2]
tcat2 = proc_testX[:, 3]

emb_model.fit([continous, cat1, cat2], trainY, epochs=20, batch_size=16, validation_data=([tcontinous, tcat1, tcat2], testY))

Train on 8000 samples, validate on 2000 samples
Epoch 1/20
8000/8000 [==============================] - 1s 66us/step - loss: 0.6604 - acc: 0.6054 - val_loss: 0.5512 - val_acc: 0.7290
Epoch 2/20
8000/8000 [==============================] - 0s 48us/step - loss: 0.4517 - acc: 0.7441 - val_loss: 0.3829 - val_acc: 0.8125
Epoch 3/20
8000/8000 [==============================] - 0s 49us/step - loss: 0.3367 - acc: 0.8660 - val_loss: 0.3016 - val_acc: 0.8645
Epoch 4/20
8000/8000 [==============================] - 0s 49us/step - loss: 0.2770 - acc: 0.8774 - val_loss: 0.2568 - val_acc: 0.8710
Epoch 5/20
8000/8000 [==============================] - 0s 51us/step - loss: 0.2364 - acc: 0.8907 - val_loss: 0.2141 - val_acc: 0.9045
Epoch 6/20
8000/8000 [==============================] - 0s 51us/step - loss: 0.1928 - acc: 0.9220 - val_loss: 0.1736 - val_acc: 0.9410
Epoch 7/20
8000/8000 [==============================] - 0s 49us/step - loss: 0.1536 - acc: 0.9506 - val_loss: 0.1400 - val_acc: 0.9610
Epoch 8

# Evaluation

In [27]:
def evaluate(integer=True, embed=False, ep=10, function='linear'):
    for i in range(2, 9):
        
        data = pd.read_csv(str(function) + '_1000_' + str(i) + '.csv') 
        df = pd.DataFrame(data)
        
        (trainX, valX) = train_test_split(df, test_size=0.2, random_state=42)
        trainY = trainX['picked_node']
        valY = valX['picked_node']
        
        if integer:
            (proc_trainX, proc_valX) = process_integer_data(df, trainX, valX)
        else:
            (proc_trainX, proc_valX) = process_structured_data(df, trainX, valX)
        
        if embed:
            model = emb(len(np.unique(proc_trainX[:, 2])))
            continous = proc_trainX[:, :2]
            cat1 = proc_trainX[:, 2]
            cat2 = proc_trainX[:, 3]

            vcontinous = proc_valX[:, :2]
            vcat1 = proc_valX[:, 2]
            vcat2 = proc_valX[:, 3]

            model.fit([continous, cat1, cat2], epochs=ep, batch_size=16, validation_data=([vcontinous, vcat1, vcat2], valY))
        else:
            model = create_mlp(proc_trainX.shape[1])
            model.fit(proc_trainX, trainY, batch_size=10, epochs=ep, validation_data=(proc_valX, valY))
            
        data = pd.read_csv(str(function) + '_200_' + str(i) + '.csv') 
        df= pd.DataFrame(data)
        
        (trainX, testX) = train_test_split(df, test_size=0.01, random_state=42)
        trainY = trainX['picked_node']
        testY = testX['picked_node']
        
        if integer:
            (proc_testX, proc_test2X) = process_integer_data(df, trainX, valX)
        else:
            (proc_testX, proc_test2X) = process_structured_data(df, trainX, valX)
        
        
        _, accuracy = model.evaluate(proc_testX, trainY, batch_size=128)
        
        print('#####################################################\n\n')
        print(str(i) + " colours accuracy = " + str(accuracy))
        print('\n\n#####################################################')

evaluate(integer=False, ep=10,function='non-trans')
        
        
        

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 0s 158us/step - loss: 0.7124 - accuracy: 0.4400 - val_loss: 0.6855 - val_accuracy: 0.5200
Epoch 2/10
800/800 [==============================] - 0s 64us/step - loss: 0.6574 - accuracy: 0.5863 - val_loss: 0.6343 - val_accuracy: 0.5800
Epoch 3/10
800/800 [==============================] - 0s 60us/step - loss: 0.6094 - accuracy: 0.7788 - val_loss: 0.5829 - val_accuracy: 0.8500
Epoch 4/10
800/800 [==============================] - 0s 64us/step - loss: 0.5599 - accuracy: 0.8325 - val_loss: 0.5320 - val_accuracy: 0.8500
Epoch 5/10
800/800 [==============================] - 0s 66us/step - loss: 0.5110 - accuracy: 0.8325 - val_loss: 0.4823 - val_accuracy: 0.8500
Epoch 6/10
800/800 [==============================] - 0s 64us/step - loss: 0.4650 - accuracy: 0.8350 - val_loss: 0.4377 - val_accuracy: 0.8550
Epoch 7/10
800/800 [==============================] - 0s 64us/step - loss: 0.4233 - accuracy: 0

198/198 [==============================] - 0s 10us/step
#####################################################


6 colours accuracy = 0.7676767706871033


#####################################################
Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 0s 162us/step - loss: 0.7246 - accuracy: 0.5150 - val_loss: 0.7307 - val_accuracy: 0.5200
Epoch 2/10
800/800 [==============================] - 0s 65us/step - loss: 0.7112 - accuracy: 0.5075 - val_loss: 0.7195 - val_accuracy: 0.5000
Epoch 3/10
800/800 [==============================] - 0s 72us/step - loss: 0.7004 - accuracy: 0.4750 - val_loss: 0.7094 - val_accuracy: 0.4800
Epoch 4/10
800/800 [==============================] - 0s 82us/step - loss: 0.6899 - accuracy: 0.4863 - val_loss: 0.7000 - val_accuracy: 0.4850
Epoch 5/10
800/800 [==============================] - 0s 65us/step - loss: 0.6786 - accuracy: 0.5350 - val_loss: 0.6899 - val_accuracy: 0.5100
Epoch 6/10
800/800 [==========